In [2]:
import keras
from keras.layers import Activation, Dense, Dropout, Conv2D, \
                         Flatten, MaxPooling2D
from keras.models import Sequential
import librosa
import librosa.display
import numpy as np
import pandas as pd
import random
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
arr = np.random.randint(0, 10, (7, 4))

def padding(array, xx, yy):
    """
    :param array: numpy array
    :param xx: desired height
    :param yy: desirex width
    :return: padded array
    """

    h = array.shape[0]
    w = array.shape[1]

    a = (xx - h) // 2
    aa = xx - a - h

    b = (yy - w) // 2
    bb = yy - b - w

    return np.pad(array, pad_width=((a, aa), (b, bb)), mode='constant')

In [4]:
import os
import numpy as np
from tqdm import tqdm

D = [] # Dataset
data = {
        "spect": [],
        "label": []
       }

directories =[  
                 ['F01','Atypical'],['F03','Atypical'],['F04','Atypical'],['FC01','Typical'],['FC02','Typical'],
                 ['FC03','Typical'],['M01','Atypical'],['M02','Atypical'],['M03','Atypical'],['M04','Atypical'],
                 ['M05','Atypical'],['MC01','Typical'],['MC02','Typical'],['MC03','Typical'],['MC04','Typical']
             ]

for pair in directories:
    directory = pair[0]
    label = pair[1]
    print(directory)
    for filename in os.listdir(directory):
        if filename != 'Notes':
            file = os.path.join(directory, filename)
            file = os.path.join(file, 'wav_arrayMic')
            if os.path.isdir(file):
                for audiofile in tqdm(os.listdir(file)):
                    f = os.path.join(file, audiofile)
                    if os.path.isfile(f):
                        if librosa.get_duration(filename=f) != 0:
                            y, sr = librosa.load(f)
                            
                            #y_changed = librosa.effects.time_stretch(y, rate=0.81)
                            y_changed = librosa.effects.pitch_shift(y, sr, n_steps=-2.5)
                            ps = librosa.feature.melspectrogram(y=y_changed, sr=sr)
                            
                            if ps.shape[1] > 400 : continue
                            PS = ps
                            data["spect"].append(PS.tolist())
                            data["label"].append(label)
                            D.append( (np.array(padding(ps,128,400)), label) )

  0%|                                                                                          | 0/134 [00:00<?, ?it/s]

F01


  0%|                                                                                          | 0/204 [00:00<?, ?it/s]

F03


  0%|                                                                                          | 0/199 [00:00<?, ?it/s]

...

  0%|                                                                                          | 0/600 [00:00<?, ?it/s]

MC03


  0%|                                                                                          | 0/648 [00:00<?, ?it/s]

MC04


100%|████████████████████████████████████████████████████████████████████████████████| 373/373 [00:51<00:00,  7.18it/s]


In [5]:
dataset = D
random.shuffle(dataset)

train = dataset[:8000]
dataset = dataset[8000:]
validate = dataset[:1000]
test = dataset[1000:]

X_train, y_train = zip(*train)
X_validate, y_validate = zip(*validate)
X_test, y_test = zip(*test)

# Reshape for CNN input
X_train = np.array([x.reshape( (128, 400, 1) ) for x in X_train])
X_validate = np.array([x.reshape( (128, 400, 1) ) for x in X_validate])
X_test = np.array([x.reshape( (128, 400, 1) ) for x in X_test])

In [6]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
#y=to_categorical(labelencoder.fit_transform(y))
#y
y_train = np.array(to_categorical(labelencoder.fit_transform(y_train)))
y_validate = np.array(to_categorical(labelencoder.fit_transform(y_validate)))
y_test = np.array(to_categorical(labelencoder.fit_transform(y_test)))

In [7]:
X_train.shape, X_validate.shape, X_test.shape, y_train.shape, y_validate.shape, y_test.shape

((8000, 128, 400, 1),
 (1000, 128, 400, 1),
 (238, 128, 400, 1),
 (8000, 2),
 (1000, 2),
 (238, 2))

In [8]:
model = Sequential()
input_shape=(128, 400, 1)

model.add(Conv2D(24, (5, 5), strides=(1, 1), input_shape=input_shape))
model.add(MaxPooling2D((4, 2), strides=(4, 2)))
model.add(Activation('relu'))

model.add(Conv2D(48, (5, 5), padding="valid"))
model.add(MaxPooling2D((4, 2), strides=(4, 2)))
model.add(Activation('relu'))

model.add(Conv2D(48, (5, 5), padding="valid"))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dropout(rate=0.5))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(rate=0.5))

model.add(Dense(2))
model.add(Activation('softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 124, 396, 24)      624       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 198, 24)       0         
_________________________________________________________________
activation (Activation)      (None, 31, 198, 24)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 194, 48)       28848     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 97, 48)         0         
_________________________________________________________________
activation_1 (Activation)    (None, 6, 97, 48)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 2, 93, 48)         5

In [9]:
model.compile(optimizer="Adam",loss="categorical_crossentropy",metrics=['accuracy'])
model.fit(x=X_train, y=y_train,epochs=12,batch_size=128,validation_data= (X_validate, y_validate))
score = model.evaluate(x=X_test,y=y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/12
63/63 [==============================] - 298s 5s/step - loss: 0.5745 - accuracy: 0.7685 - val_loss: 0.4430 - val_accuracy: 0.8260
Epoch 2/12
63/63 [==============================] - 299s 5s/step - loss: 0.4236 - accuracy: 0.8289 - val_loss: 0.4142 - val_accuracy: 0.8380
Epoch 3/12
63/63 [==============================] - 290s 5s/step - loss: 0.4034 - accuracy: 0.8385 - val_loss: 0.4075 - val_accuracy: 0.8410
Epoch 4/12
63/63 [==============================] - 287s 5s/step - loss: 0.3851 - accuracy: 0.8462 - val_loss: 0.4029 - val_accuracy: 0.8500
Epoch 5/12
63/63 [==============================] - 288s 5s/step - loss: 0.3689 - accuracy: 0.8537 - val_loss: 0.3950 - val_accuracy: 0.8650
Epoch 6/12
63/63 [==============================] - 286s 5s/step - loss: 0.3577 - accuracy: 0.8599 - val_loss: 0.4032 - val_accuracy: 0.8490
Epoch 7/12
63/63 [==============================] - 293s 5s/step - loss: 0.3633 - accuracy: 0.8545 - val_loss: 0.4146 - val_accuracy: 0.8570
Epoch 8/12
63

In [10]:
import json
with open('DATA_slowedPitch.json', "w") as fp:json.dump(data, fp, indent=1)

In [15]:
# save the model to disk
#filename = 'finalized_model.sav'
model.save('save_model_slowedPitch')

INFO:tensorflow:Assets written to: save_model_slowedPitch\assets


In [16]:
predict = model.predict_classes(X_test)
prediction_class = labelencoder.inverse_transform(predict)
#real_class = labelencoder.inverse_transform(y_test)
#prediction_class
x, y = zip(*test)
y
match = 0
print('  Real       Prediction       Match')
for i in range(0,X_test.shape[0]):
    if(y[i]==prediction_class[i]):
        match+=1
    print(y[i] + '       ' + prediction_class[i] + '       ' + str(y[i]==prediction_class[i]))
print('Matched = ', match)
print('Not Matched = ',(X_test.shape[0] - match))

  Real       Prediction       Match
Typical       Typical       True
Atypical       Typical       False
Typical       Typical       True
Typical       Typical       True
Atypical       Atypical       True
Typical       Atypical       False
Atypical       Atypical       True
Typical       Typical       True
Typical       Typical       True
Typical       Atypical       False
Atypical       Atypical       True
Atypical       Typical       False
Typical       Typical       True
Typical       Typical       True
Typical       Typical       True
Atypical       Atypical       True
Atypical       Atypical       True
Atypical       Atypical       True
Atypical       Typical       False
Atypical       Typical       False
Typical       Typical       True
Typical       Typical       True
Typical       Typical       True
Typical       Typical       True
Typical       Typical       True
Typical       Typical       True
Typical       Typical       True
Typical       Typical       True
Typical       Ty

In [19]:
reconstructed_model = keras.models.load_model("save_model_slowedPitch")

In [20]:
predict = reconstructed_model.predict_classes(X_test)
prediction_class = labelencoder.inverse_transform(predict)
#real_class = labelencoder.inverse_transform(y_test)
#prediction_class
x, y = zip(*test)
y
match = 0
print('  Real       Prediction       Match')
for i in range(0,X_test.shape[0]):
    if(y[i]==prediction_class[i]):
        match+=1
    print(y[i] + '       ' + prediction_class[i] + '       ' + str(y[i]==prediction_class[i]))
print('Matched = ', match)
print('Not Matched = ',(X_test.shape[0] - match))

  Real       Prediction       Match
Typical       Typical       True
Atypical       Typical       False
Typical       Typical       True
Typical       Typical       True
Atypical       Atypical       True
Typical       Atypical       False
Atypical       Atypical       True
Typical       Typical       True
Typical       Typical       True
Typical       Atypical       False
Atypical       Atypical       True
Atypical       Typical       False
Typical       Typical       True
Typical       Typical       True
Typical       Typical       True
Atypical       Atypical       True
Atypical       Atypical       True
Atypical       Atypical       True
Atypical       Typical       False
Atypical       Typical       False
Typical       Typical       True
Typical       Typical       True
Typical       Typical       True
Typical       Typical       True
Typical       Typical       True
Typical       Typical       True
Typical       Typical       True
Typical       Typical       True
Typical       Ty